In [13]:
# Libraries needed for basic web-scraping
from IPython.core.display import HTML
from bs4 import BeautifulSoup, NavigableString, Tag
from IPython.display import IFrame
from urllib.request import urlopen, Request
import pandas as pd # will use to store the data from the webpage

In [ ]:
# Scrape data from website
target_url='https://coinatmradar.com/country/226/bitcoin-atm-united-states/'
# fool the server :/
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
req = Request(url=target_url, headers=headers) 
# coinatmradar_usa_html = urlopen(req).read() 
# print(type(coinatmradar_usa_html))

# # Saving scraped HTML to .html file (for later processing)
# with open('coinatmradar_usa.html', 'w') as f:
#     f.write(coinatmradar_usa_html)

# Use html.parser to create soup
s = BeautifulSoup(coinatmradar_usa_html, 'html.parser')
print(s.prettify())


In [ ]:
cities = s.find_all('div', {'class': 'third'})
i = 0

city_venue_cnts = []
for column in cities: 
    cur_state = ''
    for elem in column:
        if isinstance(elem, NavigableString):
            continue
        if isinstance(elem, Tag):
            if 'class' in elem.attrs:
                cur_state = elem.text.split(',')[0]
                continue

            city_name = elem.a.text
            tmp_str_arr = elem.text.split(' ')
            venue_count = tmp_str_arr[len(tmp_str_arr)-1]
            try: 
                venue_count = int(venue_count)
            except ValueError as e:
                print(e)
                continue
            
            city_venue_cnt = {}
            city_venue_cnt['city']  = city_name
            city_venue_cnt['state'] = cur_state
            city_venue_cnt['count'] = venue_count
            city_venue_cnts.append(city_venue_cnt)

# remove duplicate value for Birmingham            
city_venue_cnts = city_venue_cnts[1:]
city_venue_cnts[0]['city'] = city_venue_cnts[0]['city'].split(',')[0]
df_venues_by_city = pd.json_normalize(city_venue_cnts)
# TODO: need to change this relative path when code is moved 
df_venues_by_city.to_csv('../data/interim/ATM_COUNT_BY_CTIY', index=False)

print(df_venues_by_city.shape)
for cnt in city_venue_cnts:
    print(cnt)

In [ ]:
https://www.city-data.com/

In [43]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

options = Options()
options.headless = True
options.add_argument("--window-size=1920,1200")

DRIVER_PATH = '/usr/local/bin/chromedriver'
driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)
driver.get("https://coinatmradar.com/city/1677/bitcoin-atm-south-bend/")
driver.quit()

driver.find_element_by_xpath('/html/body/div[3]/div/div/div/div[1]/h6')



WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home
